In [1]:
import torch
import torch.nn as nn

from functools import partial
from dataclasses import dataclass
from collections import OrderedDict

# Implementing ResNet in PyTorch
Today we are going to implement the famous ResNet from Kaiming He et al. (Microsoft Research). It won the 1st place on the ILSVRC 2015 classification task.

The original paper can be read from [here ](https://arxiv.org/abs/1512.03385) and it is very easy to follow, additional material can be found in this [quora answer](https://www.quora.com/)
![alt](./images/custom/rotated-resnet34.png)

*Deeper neural networks are more difficult to train.* Why? One big problem of deeper network is the vanishing gradient. Basically, the model is not able to learn anymore.

To solve this problem, the Authors proposed to use a reference to the previous layer to compute the output at a given layer. In ResNet, the output form the previous layer, called **residual**, is added to the output of the current layer. The following picture visualizes this operation

![alt](./images/residual.png)

We are going to make our implementation **as scalable as possible** using one think think unknown to mostly of the data scientiest: **object orienting programming**

## Basic Block

Okay, the first thing is to think about what we need. Well, first of all we need a convolution layer and since PyTorch does not have the 'auto' padding in Conv2d, so we have to code ourself!

In [2]:
class Conv2dAuto(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding =  (self.kernel_size[0] // 2, self.kernel_size[1] // 2) # dynamic add padding based on the kernel_size
        
conv3x3 = partial(Conv2dAuto, kernel_size=3, bias=False)      
        

In [3]:
conv = conv3x3(in_channels=32, out_channels=64)
print(conv)
del conv

Conv2dAuto(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


## Residual Block
To make clean code is mandatory to think about the main building block of each application, or of the network in our case. The residual block takes an input with `in_channels`, applies some blocks of convolutional layers to reduce it to `out_channels` and sum it up to the original input. If their sizes mismatch, then the input goes into an `identity`. We can abstract this process and create a interface that can be extedend.

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.in_channels, self.out_channels =  in_channels, out_channels
        self.blocks = nn.Identity()
        self.shortcut = nn.Identity()   
    
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut: residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels

In [5]:
ResidualBlock(32, 64)

ResidualBlock(
  (blocks): Identity()
  (shortcut): Identity()
)

Let's test it with a dummy vector with one one, we should get a vector with two

In [6]:
dummy = torch.ones((1, 1, 1, 1))

block = ResidualBlock(1, 64)
block(dummy)

tensor([[[[2.]]]])

In ResNet each block has a expansion parameter in order to increase the `out_channels`. Also, the identity is defined as a Convolution followed by an Activation layer, this is referred as `shortcut`. Then, we can just extend `ResidualBlock` and defined the `shortcut` function.

In [7]:
from collections import OrderedDict

class ResNetResidualBlock(ResidualBlock):
    def __init__(self, in_channels, out_channels, expansion=1, downsampling=1, conv=conv3x3, *args, **kwargs):
        super().__init__(in_channels, out_channels)
        self.expansion, self.downsampling, self.conv = expansion, downsampling, conv
        self.shortcut = nn.Sequential(OrderedDict(
        {
            'conv' : nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1,
                      stride=self.downsampling, bias=False),
            'bn' : nn.BatchNorm2d(self.expanded_channels)
            
        })) if self.should_apply_shortcut else None
        
        
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels

In [8]:
ResNetResidualBlock(32, 64)

ResNetResidualBlock(
  (blocks): Identity()
  (shortcut): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

### Basic Block
A basic ResNet block is composed by two layers of `3x3` convs/batchnorm/relu. In the picture, the lines represnet the residual operation. The dotted line means that the shortcut was applied to match the input and the output dimension.
![alt](./images/custom/Block.png)

Let's first create an handy function to stack one conv and batchnorm layer. Using `OrderedDict` to properly name each sublayer.

In [9]:
from collections import OrderedDict
def conv_bn(in_channels, out_channels, conv, *args, **kwargs):
    return nn.Sequential(OrderedDict({'conv': conv(in_channels, out_channels, *args, **kwargs), 
                          'bn': nn.BatchNorm2d(out_channels) }))

In [10]:
conv_bn(3, 3, nn.Conv2d, kernel_size=3)

Sequential(
  (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (bn): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [11]:
class ResNetBasicBlock(ResNetResidualBlock):
    expansion = 1
    def __init__(self, in_channels, out_channels, activation=nn.ReLU, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.blocks = nn.Sequential(
            conv_bn(self.in_channels, self.out_channels, conv=self.conv, bias=False, stride=self.downsampling),
            activation(),
            conv_bn(self.out_channels, self.expanded_channels, conv=self.conv, bias=False),
        )
    

In [12]:
dummy = torch.ones((1, 32, 224, 224))

block = ResNetBasicBlock(32, 64)
block(dummy).shape
print(block)

ResNetBasicBlock(
  (blocks): Sequential(
    (0): Sequential(
      (conv): Conv2dAuto(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU()
    (2): Sequential(
      (conv): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (shortcut): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


### BottleNeck
To increase the network deepths but to decrese the number of parameters, the Authors defined a BottleNeck block that 
"The three layers are 1x1, 3x3, and 1x1 convolutions, where the 1×1 layers are responsible for reducing and then increasing (restoring) dimensions, leaving the 3×3 layer a bottleneck with smaller input/output dimensions." We can extend the `ResNetResidualBlock` and create these blocks.

In [13]:
class ResNetBottleNeckBlock(ResNetResidualBlock):
    expansion = 4
    def __init__(self, in_channels, out_channels, activation=nn.ReLU, *args, **kwargs):
        super().__init__(in_channels, out_channels, expansion=4, *args, **kwargs)
        self.blocks = nn.Sequential(
           conv_bn(self.in_channels, self.out_channels, self.conv, kernel_size=1),
             activation(),
             conv_bn(self.out_channels, self.out_channels, self.conv, kernel_size=3, stride=self.downsampling),
             activation(),
             conv_bn(self.out_channels, self.expanded_channels, self.conv, kernel_size=1),
        )
    

In [14]:
dummy = torch.ones((1, 32, 10, 10))

block = ResNetBottleNeckBlock(32, 64)
block(dummy).shape
print(block)

ResNetBottleNeckBlock(
  (blocks): Sequential(
    (0): Sequential(
      (conv): Conv2dAuto(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU()
    (2): Sequential(
      (conv): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): ReLU()
    (4): Sequential(
      (conv): Conv2dAuto(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (shortcut): Sequential(
    (conv): Conv2d(32, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


### Layer
A ResNet's layer is composed by blocks stacked one after the other. 

![alt](./images/custom/Layer.png)

We can easily defined it by just stuck `n` blocks one after the other, just remember that the first convolution block has a stide of two since "We perform downsampling directly by convolutional layers that have a stride of 2".

In [15]:
class ResNetLayer(nn.Module):
    def __init__(self, in_channels, out_channels, block=ResNetBasicBlock, n=1, *args, **kwargs):
        super().__init__()
        # 'We perform downsampling directly by convolutional layers that have a stride of 2.'
        downsampling = 2 if in_channels != out_channels else 1
        
        self.blocks = nn.Sequential(
            block(in_channels , out_channels, *args, **kwargs, downsampling=downsampling),
            *[block(out_channels * block.expansion, 
                    out_channels, downsampling=1, *args, **kwargs) for _ in range(n - 1)]
        )

    def forward(self, x):
        x = self.blocks(x)
        return x

In [16]:
dummy = torch.ones((1, 32, 48, 48))

layer = ResNetLayer(64, 128, block=ResNetBasicBlock, n=3)
# layer(dummy).shape
layer

ResNetLayer(
  (blocks): Sequential(
    (0): ResNetBasicBlock(
      (blocks): Sequential(
        (0): Sequential(
          (conv): Conv2dAuto(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): ReLU()
        (2): Sequential(
          (conv): Conv2dAuto(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (shortcut): Sequential(
        (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ResNetBasicBlock(
      (blocks): Sequential(
        (0): Sequential(
          (conv): Conv2dAuto(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn

### Encoder
Similarly, the encoder is composed by multiple layer at increasing features size.

![alt](./images/custom/rotated-Encoder.png)



In [17]:
class ResNetEncoder(nn.Module):
    """
    ResNet encoder composed by increasing different layers with increasing features.
    """
    def __init__(self, in_channels=3, blocks_sizes=[64, 128, 256, 512], deepths=[2,2,2,2], 
                 activation=nn.ReLU, block=ResNetBasicBlock, *args,**kwargs):
        super().__init__()
        
        self.blocks_sizes = blocks_sizes
        
        self.gate = nn.Sequential(
            nn.Conv2d(in_channels, self.blocks_sizes[0], kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(self.blocks_sizes[0]),
            activation(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.in_out_block_sizes = list(zip(blocks_sizes, blocks_sizes[1:]))
        self.blocks = nn.ModuleList([ 
            ResNetLayer(blocks_sizes[0], blocks_sizes[0], n=deepths[0], activation=activation, 
                        block=block,  *args, **kwargs),
            *[ResNetLayer(in_channels * block.expansion, 
                          out_channels, n=n, activation=activation, 
                          block=block, *args, **kwargs) 
              for (in_channels, out_channels), n in zip(self.in_out_block_sizes, deepths[1:])]       
        ])
        
        
    def forward(self, x):
        x = self.gate(x)
        for block in self.blocks:
            x = block(x)
        return x

## Decoder
The decoder is the last piece we need to create the full network. It is a fully connected layer that maps the features learned by the network to their respective classes. Easily, we can defined it as:

In [18]:
class ResnetDecoder(nn.Module):
    """
    This class represents the tail of ResNet. It performs a global pooling and maps the output to the
    correct class by using a fully connected layer.
    """
    def __init__(self, in_features, n_classes):
        super().__init__()
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.decoder = nn.Linear(in_features, n_classes)

    def forward(self, x):
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        x = self.decoder(x)
        return x


## ResNet

Final, we can put all the pieces together and create the final model.

![alt](./images/custom/rotated-resnet34.png)

In [38]:
class ResNet(nn.Module):
    
    def __init__(self, in_channels, n_classes, *args, **kwargs):
        super().__init__()
        self.encoder = ResNetEncoder(in_channels, *args, **kwargs).to(device)
        self.decoder = ResnetDecoder(self.encoder.blocks[-1].blocks[-1].expanded_channels, n_classes).to(device)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

We can now defined the five models proposed by the Authors, `resnet18,34,50,101,152`

In [39]:
def resnet18(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBasicBlock, deepths=[2, 2, 2, 2])

def resnet34(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBasicBlock, deepths=[3, 4, 6, 3])

def resnet50(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBottleNeckBlock, deepths=[3, 4, 6, 3])

def resnet101(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBottleNeckBlock, deepths=[3, 4, 23, 3])

def resnet152(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBottleNeckBlock, deepths=[3, 8, 36, 3])

In [35]:
from torchsummary import summary

model = resnet101(3, 1000)
summary(model.cuda(), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5          [-1, 256, 56, 56]          16,384
       BatchNorm2d-6          [-1, 256, 56, 56]             512
        Conv2dAuto-7           [-1, 64, 56, 56]           4,096
       BatchNorm2d-8           [-1, 64, 56, 56]             128
              ReLU-9           [-1, 64, 56, 56]               0
       Conv2dAuto-10           [-1, 64, 56, 56]          36,864
      BatchNorm2d-11           [-1, 64, 56, 56]             128
             ReLU-12           [-1, 64, 56, 56]               0
       Conv2dAuto-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [22]:
import torchvision.models as models

# resnet101(False)

summary(models.resnet101(False).cuda(), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [22]:
import os
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import tqdm

In [23]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
def loadNpy(filename):
    with open(filename, "rb") as f: return np.load(f)

db = f"{os.getcwd()}/../../data/slices/gaussian-grabber/size160000-hw40-d100/"
data_type = ""

In [25]:
X_train, X_val, X_test = loadNpy(db + f"X_train.npy"), loadNpy(db + f"X_val.npy"), loadNpy(db + f"X_test.npy")
y_train, y_val, y_test = loadNpy(db + f"c-y{data_type}_train.npy"), loadNpy(db + f"c-y{data_type}_val.npy"), loadNpy(db + f"c-y{data_type}_test.npy")

In [26]:
def npy_to_tensor(data, l=False):
    if not l:
        data = torch.from_numpy(data).float()
    else:
        data = torch.from_numpy(data).type(torch.LongTensor)
    return data

In [27]:
X_train, X_val, X_test = npy_to_tensor(X_train), npy_to_tensor(X_val), npy_to_tensor(X_test)
y_train, y_val, y_test = npy_to_tensor(y_train, 1), npy_to_tensor(y_val, 1), npy_to_tensor(y_test, 1)

In [28]:
batch_size = 64

In [29]:
train_dataset = TensorDataset(X_train, y_train)

val_dataset = TensorDataset(X_val, y_val)

test_dataset = TensorDataset(X_test, y_test)

# Data loader
train_iterator = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=False)

val_iterator = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

test_iterator = DataLoader(dataset=test_dataset, batch_size=batch_size,shuffle=False)

In [53]:
loaders = {
    'train': train_iterator,
    'val': val_iterator,
    'test': test_iterator
}

In [44]:
import torch.optim as optim

In [92]:
rn = resnet34(40, 2)

In [93]:
loss_func = nn.CrossEntropyLoss()   
loss_func

CrossEntropyLoss()

In [94]:
optimizer = optim.Adam(rn.parameters(), lr = 0.001, weight_decay=1e-3)   
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0.001
)

In [ ]:
num_epochs = 1000
nn_epochs = {}

# Train the model
total_step = len(train_iterator)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_iterator):  
        # Move tensors to the configured device
        #print(images.shape)
        images = images.reshape(-1, 40,40,100).to(device)
        labels = labels.to(device)
        
        # Forward pass
        #print(images.shape)
        outputs = rn(images)
        loss = loss_func(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print(i)

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    nn_epochs[epoch] = {"loss": loss.item(), "train": eval_rn(rn, "train"), 
                 "val": eval_rn(rn, "val"), "test": eval_rn(rn, "test")}
    rn.train()

Epoch [1/1000], Step [19/19], Loss: 0.5427
train Accuracy of the model on the 1200 train images: 0.846
val Accuracy of the model on the 400 val images: 0.868
test Accuracy of the model on the 400 test images: 0.843
Epoch [2/1000], Step [19/19], Loss: 0.4840
train Accuracy of the model on the 1200 train images: 0.846
val Accuracy of the model on the 400 val images: 0.868
test Accuracy of the model on the 400 test images: 0.843
Epoch [3/1000], Step [19/19], Loss: 0.4315
train Accuracy of the model on the 1200 train images: 0.848
val Accuracy of the model on the 400 val images: 0.868
test Accuracy of the model on the 400 test images: 0.843
Epoch [4/1000], Step [19/19], Loss: 0.3827
train Accuracy of the model on the 1200 train images: 0.848
val Accuracy of the model on the 400 val images: 0.865
test Accuracy of the model on the 400 test images: 0.840
Epoch [5/1000], Step [19/19], Loss: 0.4294
train Accuracy of the model on the 1200 train images: 0.851
val Accuracy of the model on the 400 

Epoch [39/1000], Step [19/19], Loss: 0.0067
train Accuracy of the model on the 1200 train images: 0.973
val Accuracy of the model on the 400 val images: 0.843
test Accuracy of the model on the 400 test images: 0.833
Epoch [40/1000], Step [19/19], Loss: 0.0011
train Accuracy of the model on the 1200 train images: 0.991
val Accuracy of the model on the 400 val images: 0.772
test Accuracy of the model on the 400 test images: 0.767
Epoch [41/1000], Step [19/19], Loss: 0.0917
train Accuracy of the model on the 1200 train images: 0.990
val Accuracy of the model on the 400 val images: 0.810
test Accuracy of the model on the 400 test images: 0.800
Epoch [42/1000], Step [19/19], Loss: 0.0046
train Accuracy of the model on the 1200 train images: 0.998
val Accuracy of the model on the 400 val images: 0.802
test Accuracy of the model on the 400 test images: 0.800
Epoch [43/1000], Step [19/19], Loss: 0.0008
train Accuracy of the model on the 1200 train images: 0.997
val Accuracy of the model on the

Epoch [77/1000], Step [19/19], Loss: 0.0000
train Accuracy of the model on the 1200 train images: 0.999
val Accuracy of the model on the 400 val images: 0.845
test Accuracy of the model on the 400 test images: 0.820
Epoch [78/1000], Step [19/19], Loss: 0.0000
train Accuracy of the model on the 1200 train images: 0.999
val Accuracy of the model on the 400 val images: 0.848
test Accuracy of the model on the 400 test images: 0.820
Epoch [79/1000], Step [19/19], Loss: 0.0000
train Accuracy of the model on the 1200 train images: 0.999
val Accuracy of the model on the 400 val images: 0.845
test Accuracy of the model on the 400 test images: 0.812
Epoch [80/1000], Step [19/19], Loss: 0.0000
train Accuracy of the model on the 1200 train images: 0.999
val Accuracy of the model on the 400 val images: 0.845
test Accuracy of the model on the 400 test images: 0.812
Epoch [81/1000], Step [19/19], Loss: 0.0000
train Accuracy of the model on the 1200 train images: 0.999
val Accuracy of the model on the

Epoch [115/1000], Step [19/19], Loss: 0.0910
train Accuracy of the model on the 1200 train images: 0.978
val Accuracy of the model on the 400 val images: 0.838
test Accuracy of the model on the 400 test images: 0.820
Epoch [116/1000], Step [19/19], Loss: 0.0437
train Accuracy of the model on the 1200 train images: 0.987
val Accuracy of the model on the 400 val images: 0.840
test Accuracy of the model on the 400 test images: 0.820
Epoch [117/1000], Step [19/19], Loss: 0.0048
train Accuracy of the model on the 1200 train images: 0.986
val Accuracy of the model on the 400 val images: 0.843
test Accuracy of the model on the 400 test images: 0.823
Epoch [118/1000], Step [19/19], Loss: 0.0079
train Accuracy of the model on the 1200 train images: 0.996
val Accuracy of the model on the 400 val images: 0.835
test Accuracy of the model on the 400 test images: 0.815
Epoch [119/1000], Step [19/19], Loss: 0.0009
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model o

Epoch [153/1000], Step [19/19], Loss: 0.0011
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.843
test Accuracy of the model on the 400 test images: 0.828
Epoch [154/1000], Step [19/19], Loss: 0.0008
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.853
test Accuracy of the model on the 400 test images: 0.833
Epoch [155/1000], Step [19/19], Loss: 0.0007
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.868
test Accuracy of the model on the 400 test images: 0.838
Epoch [156/1000], Step [19/19], Loss: 0.0007
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.868
test Accuracy of the model on the 400 test images: 0.840
Epoch [157/1000], Step [19/19], Loss: 0.0008
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model o

Epoch [191/1000], Step [19/19], Loss: 0.0005
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.855
test Accuracy of the model on the 400 test images: 0.845
Epoch [192/1000], Step [19/19], Loss: 0.0004
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.855
test Accuracy of the model on the 400 test images: 0.845
Epoch [193/1000], Step [19/19], Loss: 0.0004
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.855
test Accuracy of the model on the 400 test images: 0.845
Epoch [194/1000], Step [19/19], Loss: 0.0004
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.850
test Accuracy of the model on the 400 test images: 0.848
Epoch [195/1000], Step [19/19], Loss: 0.0004
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model o

Epoch [229/1000], Step [19/19], Loss: 0.0006
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.812
test Accuracy of the model on the 400 test images: 0.825
Epoch [230/1000], Step [19/19], Loss: 0.0006
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.815
test Accuracy of the model on the 400 test images: 0.823
Epoch [231/1000], Step [19/19], Loss: 0.0006
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.807
test Accuracy of the model on the 400 test images: 0.825
Epoch [232/1000], Step [19/19], Loss: 0.0006
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.815
test Accuracy of the model on the 400 test images: 0.828
Epoch [233/1000], Step [19/19], Loss: 0.0007
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model o

Epoch [267/1000], Step [19/19], Loss: 0.0007
train Accuracy of the model on the 1200 train images: 0.999
val Accuracy of the model on the 400 val images: 0.840
test Accuracy of the model on the 400 test images: 0.835
Epoch [268/1000], Step [19/19], Loss: 0.5122
train Accuracy of the model on the 1200 train images: 0.718
val Accuracy of the model on the 400 val images: 0.735
test Accuracy of the model on the 400 test images: 0.723
Epoch [269/1000], Step [19/19], Loss: 0.4764
train Accuracy of the model on the 1200 train images: 0.841
val Accuracy of the model on the 400 val images: 0.853
test Accuracy of the model on the 400 test images: 0.833
Epoch [270/1000], Step [19/19], Loss: 0.4798
train Accuracy of the model on the 1200 train images: 0.846
val Accuracy of the model on the 400 val images: 0.868
test Accuracy of the model on the 400 test images: 0.843
Epoch [271/1000], Step [19/19], Loss: 0.3683
train Accuracy of the model on the 1200 train images: 0.847
val Accuracy of the model o

Epoch [305/1000], Step [19/19], Loss: 0.0004
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.850
test Accuracy of the model on the 400 test images: 0.825
Epoch [306/1000], Step [19/19], Loss: 0.0004
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.850
test Accuracy of the model on the 400 test images: 0.830
Epoch [307/1000], Step [19/19], Loss: 0.0004
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.850
test Accuracy of the model on the 400 test images: 0.823
Epoch [308/1000], Step [19/19], Loss: 0.0004
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model on the 400 val images: 0.850
test Accuracy of the model on the 400 test images: 0.830
Epoch [309/1000], Step [19/19], Loss: 0.0004
train Accuracy of the model on the 1200 train images: 1.000
val Accuracy of the model o

In [91]:
num_epochs = 10
nn_epochs = {}

# Train the model
total_step = len(val_iterator)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(val_iterator):  
        # Move tensors to the configured device
        #print(images.shape)
        images = images.reshape(-1, 40,40,100).to(device)
        labels = labels.to(device)
        
        # Forward pass
        #print(images.shape)
        outputs = rn(images)
        loss = loss_func(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print(i)

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    nn_epochs[epoch] = {"loss": loss.item(), #"train": eval_rn(rn, "train"), 
                 "val": eval_rn(rn, "val"), "test": eval_rn(rn, "test")}
    rn.train()

Epoch [1/10], Step [7/7], Loss: 0.4246
val Accuracy of the model on the 400 val images: 0.868
test Accuracy of the model on the 400 test images: 0.840
Epoch [2/10], Step [7/7], Loss: 0.3940
val Accuracy of the model on the 400 val images: 0.877
test Accuracy of the model on the 400 test images: 0.835
Epoch [3/10], Step [7/7], Loss: 0.2230
val Accuracy of the model on the 400 val images: 0.873
test Accuracy of the model on the 400 test images: 0.777
Epoch [4/10], Step [7/7], Loss: 0.1670
val Accuracy of the model on the 400 val images: 0.870
test Accuracy of the model on the 400 test images: 0.760
Epoch [5/10], Step [7/7], Loss: 0.0891
val Accuracy of the model on the 400 val images: 0.912
test Accuracy of the model on the 400 test images: 0.762
Epoch [6/10], Step [7/7], Loss: 0.1003
val Accuracy of the model on the 400 val images: 0.955
test Accuracy of the model on the 400 test images: 0.765
Epoch [7/10], Step [7/7], Loss: 0.1295
val Accuracy of the model on the 400 val images: 0.905


In [65]:
def eval_rn(rn_mod, data):
    
    rn_mod.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        pred_ys = []
        labs = []
        for images, labels in loaders[data]:
            images = images.reshape(-1,40,40,100).to(device)
            labels = labels.to(device)
            #print(rn_mod(images))
            test_output = rn_mod(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            pred_ys = pred_ys + test_output.flatten().tolist()
            labs = labs + labels.tolist()
            total += len(labels)
            correct += (pred_y == labels).sum().item()
            pass
        print(f'{data} Accuracy of the model on the {total} {data} images: %.3f' % (correct/total))
        return correct/total #, pred_ys #pred_ys, labs, correct/total
    pass